# Working with DuckDB in Juypter 
A very quick guide to getting setup with DuckDB in Jupyter. This document quickly covers using [DuckDB](https://duckdb.org/) with the [magic_duckdb](https://github.com/iqmo-org/magic_duckdb) cell magic extension.

Reference information:
- [DuckDB](https://duckdb.org/)
  - [Friendly SQL](https://duckdb.org/docs/sql/dialect/friendly_sql.html)
    - [DESCRIBE](https://duckdb.org/docs/guides/meta/describe.html)
    - [SUMMARIZE](https://duckdb.org/docs/guides/meta/summarize.html)
    - [FROM first syntax](https://duckdb.org/docs/sql/query_syntax/from.html#from-first-syntax)
    - [Directly querying CSV files](https://duckdb.org/docs/data/csv/overview.html)
    - [Metadata Functions](https://duckdb.org/docs/sql/meta/duckdb_table_functions)
      - [duckdb_databases()](https://duckdb.org/docs/sql/meta/duckdb_table_functions#duckdb_databases)
- [IPython Built-in magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html)
  - [xmode](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-xmode)
  - [reload_ext](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-reload_ext)
  - [load_ext](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-load_ext)
- [magic_duckdb](https://github.com/iqmo-org/magic_duckdb)
  - [Usage details](https://github.com/iqmo-org/magic_duckdb?tab=readme-ov-file#usage-details)

## Install our data analysis tools

We only need to run the following once and then comment it out by adding `#` before `%pip`

You might need to restart the kernel if packages were updated.

In [ ]:
%pip install duckdb magic_duckdb jupysql duckdb-engine pandas --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: C:\Users\dwayn\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


We can load our packages / extension for using DuckDB via:
`%reload_ext magic_duckdb`

I use `reload_ext` instead of `load_ext` so that if I need to rerun a cell it doesn't give me an error. 
I use `xmode minimal` so that I don't get the long-drawn-out error messages and just what is relevant.

In [3]:
%xmode minimal 
%reload_ext magic_duckdb

Exception reporting mode: Minimal


If I just want to use an in-memory DuckDB database we can just execute a query using either `%dql` or `%%dql`.

In [5]:
%dql SELECT version(); -- This will report the version of duckdb being used.

,"""version""()"
0,v1.1.3


In [6]:
%dql CALL duckdb_databases(); -- This will list the databases

,database_name,database_oid,path,comment,tags,internal,type,readonly
0,memory,1146,None,None,{},False,duckdb,False
1,system,0,None,None,{},True,duckdb,False
2,temp,1820,None,None,{},True,duckdb,False


In [7]:
%dql FROM summer_medals.csv; -- We can read directly from a csv file. 

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,HAJOS Alfred,HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,HERSCHMANN Otto,AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,DRIVAS Dimitrios,GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,MALOKINIS Ioannis,GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,CHASAPIS Spiridon,GRE,Men,100M Freestyle For Sailors,Silver
...,...,...,...,...,...,...,...,...,...
31160,2012,London,Wrestling,Wrestling Freestyle,JANIKOWSKI Damian,POL,Men,Wg 84 KG,Bronze
31161,2012,London,Wrestling,Wrestling Freestyle,REZAEI Ghasem Gholamreza,IRI,Men,Wg 96 KG,Gold
31162,2012,London,Wrestling,Wrestling Freestyle,TOTROV Rustam,RUS,Men,Wg 96 KG,Silver
31163,2012,London,Wrestling,Wrestling Freestyle,ALEKSANYAN Artur,ARM,Men,Wg 96 KG,Bronze


In [ ]:
%dql SUMMARIZE(FROM summer_medals.csv); -- Get some stats about each column

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,Year,BIGINT,1896,2012,27,1970.482785175678,33.15841604997222,1949,1980,1999,31165,0.00
1,City,VARCHAR,Amsterdam,Tokyo,23,None,None,None,None,None,31165,0.00
2,Sport,VARCHAR,Aquatics,Wrestling,42,None,None,None,None,None,31165,0.00
3,Discipline,VARCHAR,Archery,Wrestling Gre-R,57,None,None,None,None,None,31165,0.00
4,Athlete,VARCHAR,AABYE Edgar,ÖSTRAND Per-Olof,26460,None,None,None,None,None,31165,0.00
5,Country,VARCHAR,AFG,ZZX,146,None,None,None,None,None,31165,0.01
6,Gender,VARCHAR,Men,Women,2,None,None,None,None,None,31165,0.00
7,Event,VARCHAR,+ 100KG,Épée Team,535,None,None,None,None,None,31165,0.00
8,Medal,VARCHAR,Bronze,Silver,3,None,None,None,None,None,31165,0.00


In [ ]:
%%dql
-- We can create our and load data into our database table
CREATE OR REPLACE TABLE IF NOT EXISTS medals AS (
    FROM summer_medals.csv
);
SELECT * FROM medals LIMIT 10;

,Year,City,Sport,Discipline,Athlete,Country,Gender,Event,Medal
0,1896,Athens,Aquatics,Swimming,HAJOS Alfred,HUN,Men,100M Freestyle,Gold
1,1896,Athens,Aquatics,Swimming,HERSCHMANN Otto,AUT,Men,100M Freestyle,Silver
2,1896,Athens,Aquatics,Swimming,DRIVAS Dimitrios,GRE,Men,100M Freestyle For Sailors,Bronze
3,1896,Athens,Aquatics,Swimming,MALOKINIS Ioannis,GRE,Men,100M Freestyle For Sailors,Gold
4,1896,Athens,Aquatics,Swimming,CHASAPIS Spiridon,GRE,Men,100M Freestyle For Sailors,Silver
5,1896,Athens,Aquatics,Swimming,CHOROPHAS Efstathios,GRE,Men,1200M Freestyle,Bronze
6,1896,Athens,Aquatics,Swimming,HAJOS Alfred,HUN,Men,1200M Freestyle,Gold
7,1896,Athens,Aquatics,Swimming,ANDREOU Joannis,GRE,Men,1200M Freestyle,Silver
8,1896,Athens,Aquatics,Swimming,CHOROPHAS Efstathios,GRE,Men,400M Freestyle,Bronze
9,1896,Athens,Aquatics,Swimming,NEUMANN Paul,AUT,Men,400M Freestyle,Gold


In [17]:
%dql DESCRIBE medals; -- look at the table setup

,column_name,column_type,null,key,default,extra
0,Year,BIGINT,YES,None,None,None
1,City,VARCHAR,YES,None,None,None
2,Sport,VARCHAR,YES,None,None,None
3,Discipline,VARCHAR,YES,None,None,None
4,Athlete,VARCHAR,YES,None,None,None
5,Country,VARCHAR,YES,None,None,None
6,Gender,VARCHAR,YES,None,None,None
7,Event,VARCHAR,YES,None,None,None
8,Medal,VARCHAR,YES,None,None,None


In [18]:
%dql SUMMARIZE medals; -- gets some stats

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,Year,BIGINT,1896,2012,27,1970.482785175678,33.15841604997222,1949,1980,1999,31165,0.00
1,City,VARCHAR,Amsterdam,Tokyo,23,None,None,None,None,None,31165,0.00
2,Sport,VARCHAR,Aquatics,Wrestling,42,None,None,None,None,None,31165,0.00
3,Discipline,VARCHAR,Archery,Wrestling Gre-R,57,None,None,None,None,None,31165,0.00
4,Athlete,VARCHAR,AABYE Edgar,ÖSTRAND Per-Olof,26460,None,None,None,None,None,31165,0.00
5,Country,VARCHAR,AFG,ZZX,146,None,None,None,None,None,31165,0.01
6,Gender,VARCHAR,Men,Women,2,None,None,None,None,None,31165,0.00
7,Event,VARCHAR,+ 100KG,Épée Team,535,None,None,None,None,None,31165,0.00
8,Medal,VARCHAR,Bronze,Silver,3,None,None,None,None,None,31165,0.00


In [21]:
%%dql
-- Find out which cities hosted the summer olympics and the next two cities.
WITH hosts AS (
    SELECT DISTINCT Year, City
    FROM medals)

SELECT
    Year, City,
    LEAD(City, 1) OVER (ORDER BY Year ASC) AS Next_City,
    LEAD(City, 2) OVER (ORDER BY Year ASC) AS After_Next_City
FROM hosts
ORDER BY Year ASC
LIMIT 5;

,Year,City,Next_City,After_Next_City
0,1896,Athens,Paris,St Louis
1,1900,Paris,St Louis,London
2,1904,St Louis,London,Stockholm
3,1908,London,Stockholm,Antwerp
4,1912,Stockholm,Antwerp,Paris


In [24]:
%%dql -t show
-- We can display the table output differently as sometimes the code editor changes how the output appears
FROM medals
SELECT City, Year
LIMIT 5;


┌─────────┬───────┐
│  City   │ Year  │
│ varchar │ int64 │
├─────────┼───────┤
│ Athens  │  1896 │
│ Athens  │  1896 │
│ Athens  │  1896 │
│ Athens  │  1896 │
│ Athens  │  1896 │
└─────────┴───────┘



We can close our database connection via `%dql --close`

In [25]:
%dql --close

And verify that there isn't any open connections.

In [26]:
%dql -g